In [1]:
import requests
import time
import tqdm
import datetime

In [2]:
organization_names = ['Accio-Robotics', 'adastec', 'addverbtech', 'adinkratech', 'advanced-navigation/', 'agilexrobotics', 'Angel-Robotics', 'angsa-robotics', 'apple', 'Apptronik-Public', 'arav-jp', 'ArcheRobotics', 'arminstitute', 'ARTI-Robots', 'ascento-robotics', 'abbenterprise', 'software-asimov', 'AUBORobotics', 'Auterion', 'Avnet', 'BastianSolutionsRandD', 'BeagleSystems', 'blue-ocean-robotics', 'bluerobotics', 'bmwgroup', 'boschresearch', 'bosch-bshg-cloud', 'canonical', 'Capgemini', 'CapraRobotics', 'alxhoff/realsense_mipi_platform_driver', 'CaterpillarInc', 'CM-Labs-Simulations', 'cobalt-robots', 'mlcommons/cm4abtf', 'KABAM-Robotics', 'cogniteam', 'dpiet', 'loonwerks/AGREE', 'RedhawkSDR', 'constructiverealities', 'corvus-robotics', 'timtro/catnav2', 'cyberbotics', 'cyngn', 'autowarefoundation', 'deltascm', 'DEMATIC', 'botsandus', 'dimaag-ai', 'doosan-robotics', 'draperlaboratory', 'easymov', 'elephantrobotics', 'emlid', 'Enova-Robotics', 'EvarClab', 'fireflyautomatix', 'fluxauto', 'FormantIO', 'GeneralElectric', 'GESTALT-Robotics', 'gofore', 'gravisrobotics', 'Greenzie', 'greyorange', 'groove-x', 'hello-robot', 'HesaiTechnology', 'Hiwonder', 'husarion', 'Hydromea', 'hyperspec-ai', 'Ignitarium', 'iknowhow-sa', 'teaminaho', 'Infineon', 'innokrobotics', 'intel', 'Intermodalics', 'i3drobotics', 'jetbrainrobotics', 'Kapernikov', 'kiwicampus', 'lanneriotsw', 'leo-drive', 'LeoRover', 'LexxPluss', 'Livox-SDK', 'locusrobotics', 'loopx-ai', 'lucidvisionlabs', 'lulav', 'luxonis/', 'magazino', 'MakarenaLabs', 'mathworks', 'microsoft', 'MisoRobotics', 'MORAI-Autonomous', 'motius', 'nakai-omer', 'neobotix', 'nerian-vision', 'nobleo', 'NXP', 'NXROBO', 'ocadotechnology', 'Oceaneering', 'data-charya/OLA-Electric', 'omron-devhub', 'osrf', 'Opteran', 'isys-vision', 'pal-robotics', 'PerceptIn', 'pixelink-support', 'pixmoving-moveit', 'PlusAI', 'qbrobotics', 'RangeAero', 'RealManRobot', 'redwirespace', 'tomlogan501', 'remakeai', 'REMYRobotics', 'renesas', 'RobertShaw', 'robo-deck', 'robosharing', 'robotechvision', 'robotican', 'robotiq', 'rt-net', 'rybergrobotics-hardware', 'SahaRobotik', 'seegrid-fleet', 'sliQue-Inc', 'ST-ROBOTICS', 'StoglRobotics', 'swift-nav', 'Tangram-Vision/', 'telemething', 'Tettix/', 'TheConstructAi', 'eplane-ai', 'autowarefoundation/autoware', 'traclabs', 'tryolabs', 'ufactory', 'unboxrobotics', 'UniversalRobots', 'uvify', 'Visteon-Manaus', 'westonrobot', 'xaxxontech', 'yokogawa', '4am-robotics']

In [3]:
len(organization_names)

158

In [ ]:
import getpass
TOKEN = getpass.getpass("github auth token: ")

In [ ]:
def convert_languages_to_percent(languages):
  s = sum([v for k, v in languages.items()])
  return {k: 100*v/s for k, v in languages.items()}

def camel_case_split(s):
    idx = list(map(str.isupper, s))
    # mark change of case
    l = [0]
    for (i, (x, y)) in enumerate(zip(idx, idx[1:])):
        if x and not y:  # "Ul"
            l.append(i)
        elif not x and y:  # "lU"
            l.append(i+1)
    l.append(len(s))
    # for "lUl", index of "U" will pop twice, have to filter that
    return [s[x:y] for x, y in zip(l, l[1:]) if x < y]

def name_split(s):
  if "-" in s:
    return s.split("-")
  elif "_" in s:
    return s.split("_")
  else:
    return camel_case_split(s)

In [ ]:
BASE_LINK = "https://api.github.com/"
header = {"Authorization": f"Bearer {TOKEN}"}

In [ ]:
orgs_data = {}
for ORGANIZATION_NAME in tqdm.tqdm(organization_names):
  try:
    time.sleep(5)
    r = requests.get(BASE_LINK+"orgs/"+ORGANIZATION_NAME, headers=header)
    org_data = r.json()

    description = org_data.get("description")
    twitter = org_data.get("twitter_username")
    twitter_link = f"https://x.com/{twitter}" if twitter else None


    r = requests.get(BASE_LINK +"orgs/"+ ORGANIZATION_NAME + "/repos", headers=header)
    repos = []
    if r.ok:

      repo_data = r.json()
      # print(repo_data)
      if repo_data:
        for repo in repo_data:
          # print(repo)
          time.sleep(1)
          languages = requests.get(BASE_LINK+"repos/"+ORGANIZATION_NAME+f"/{repo['name']}/languages", headers=header).json()
          percent_languages = convert_languages_to_percent(languages)

          activity_data = requests.get(BASE_LINK+"repos/"+ORGANIZATION_NAME+f"/{repo['name']}/commits", headers=header, params={
            "since": (datetime.datetime.now()-datetime.timedelta(days=183)).isoformat(),
            "per_page": 100
          }).json()

          last_commit_data = requests.get(BASE_LINK+"repos/"+ORGANIZATION_NAME+f"/{repo['name']}/commits/HEAD", headers=header, params={
              "since": (datetime.datetime.now()-datetime.timedelta(days=183)).isoformat()
          }).json()


          last_commit_datetime = last_commit_data["commit"]["author"]["date"] if last_commit_data.get("commit") else None

          repos.append({
              "name": repo["name"],
              "stars": repo["stargazers_count"] if repo.get("stargazers_count") else 0,
              "languages": percent_languages,
              "half_year_commits": len(activity_data),
              "last_commit_date": last_commit_datetime,
              "topics": repo.get("topics") or []

          })


    orgs_data[ORGANIZATION_NAME] = {
        "description": description,
        "twitter_link": twitter_link,
        "repos": repos
    }
  except:
    import traceback
    traceback.print_exc()
    continue

  2%|▏         | 2/107 [01:47<1:19:33, 45.46s/it] Traceback (most recent call last):
  File "C:\Users\Мария\AppData\Local\Temp\ipykernel_9320\184159832.py", line 31, in <module>
    last_commit_data = requests.get(BASE_LINK+"repos/"+ORGANIZATION_NAME+f"/{repo['name']}/commits/HEAD", headers=header, params={
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Мария\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\requests\api.py", line 73, in get
    return request("get", url, params=params, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Мария\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\requests\api.py", line 59, in request
    return session.request(method=method, url=url, **kwargs)
           ^^^^^^^^^^

: 

: 

In [ ]:
import csv
with open('by_repo_results8.csv', 'w', newline='') as f:
  csvwriter = csv.writer(f, delimiter=';')
  csvwriter.writerow(["organization", "description", "twitter_link", "repo_name", "repo_stars", "repo_topics", "repo_languages", "repo_half_year_commits", "repo_last_commit_date"])

  for k in orgs_data.keys():
    org = orgs_data[k]
    csvwriter.writerow([k, org["description"], org["twitter_link"]])
    for repo in org["repos"]:
      langs = "%, ".join(map(lambda x: f"{x[0]}: {x[1]}", repo["languages"].items()))+"%",
      csvwriter.writerow([" "]*3 + [repo["name"], repo["stars"], repo["topics"], langs, repo["half_year_commits"], repo["last_commit_date"]])

In [ ]:
with open('by_org_results.csv', 'w', newline='') as f:
  csvwriter = csv.writer(f, delimiter=';')
  csvwriter.writerow(["organization", "top 3 repo", "stars_count", "languages", "ros_repos", "name_parts"])

  for k in orgs_data.keys():
    org = orgs_data[k]
    org_name = k
    top_3_repos = [r["name"] for r in sorted(org["repos"], key=lambda x: x["stars"], reverse=True)[:5]]
    stars_count = sum([r["stars"] for r in org["repos"]])
    languages = list(set([k for repo in org["repos"] for k in repo["languages"].keys()]))
    ros_repos = []
    name_parts = []
    for repo in org["repos"]:
      parts = name_split(repo["name"])
      name_parts.extend(parts)

      lower_parts = [p.lower() for p in parts]
      if "ros" in lower_parts or "ros2" in lower_parts:
        ros_repos.append(repo["name"])

    csvwriter.writerow([org_name, top_3_repos, stars_count, languages, ros_repos, name_parts])

    # csvwriter.writerow([k, org["description"], org["twitter_link"]])
    # for repo in org["repos"]:
    #   csvwriter.writerow([" "]*3 + [repo["name"], repo["stars"], repo["topics"], repo["languages"], repo["half_year_commits"], repo["last_commit_date"]])

In [ ]:
[k for repo in orgs_data["UrbanMachine"]["repos"] for k in repo["languages"].keys()]

['Python',
 'Python',
 'Python',
 'CMake',
 'C',
 'Makefile',
 'CMake',
 'Roff',
 'C++',
 'Shell',
 'Python',
 'Makefile',
 'Python']

#### Test

In [ ]:
r = requests.get(BASE_LINK+"orgs/"+ORGANIZATION_NAME)
org_data = r.json()

description = org_data.get("description")
twitter = org_data.get("twitter_username")
twitter_link = f"https://x.com/{twitter}" if twitter else None

print(f"Description:\n{description}\n"+f"\nTwitter:\n{twitter_link}" if twitter else "" )


Description:
Financing the 4th Industrial Revolution

Twitter:
https://x.com/MerkleB0t


In [ ]:
r = requests.get(BASE_LINK+"repos/leggedrobotics/darknet_ros", headers=header)
r.json()

In [ ]:
r = requests.get(BASE_LINK +"orgs/"+ ORGANIZATION_NAME + "/repos")
orgs = []
for org in r.json():
  languages = requests.get(BASE_LINK+"repos/"+ORGANIZATION_NAME+f"/{org['name']}/languages").json()
  percent_languages = convert_languages_to_percent(languages)

  activity_data = requests.get(BASE_LINK+"repos/"+ORGANIZATION_NAME+f"/{org['name']}/commits", params={
    "since": (datetime.datetime.now()-datetime.timedelta(days=183)).isoformat(),
    "per_page": 100
  }).json()

  last_commit_data = requests.get(BASE_LINK+"repos/"+ORGANIZATION_NAME+f"/{org['name']}/commits/HEAD", params={
      "since": (datetime.datetime.now()-datetime.timedelta(days=183)).isoformat()
  }).json()


  last_commit_datetime = last_commit_data["commit"]["author"]["date"] if last_commit_data.get("commit") else None

  orgs.append({
      "name": org["name"],
      "stars": org["stargazers_count"] if org.get("stargazers_count") else 0,
      "languages": "%, ".join(map(lambda x: f"{x[0]}: {x[1]}", percent_languages.items()))+"%",
      "half_year_commits": len(activity_data),
      "last_commit_date": last_commit_datetime

  })

In [ ]:
for org in orgs:
  print(f"Name: {org['name']}")
  print(f"Stars: {org['stars']}")
  print(f"Languages: {org['languages']}")
  print(f"Half Year Commits: {org['half_year_commits']}")
  print(f"Date of last commit: {org['last_commit_date']}")
  print("\n-----------------------------------------\n")